In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import random
import time
import os
#os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2
from tqdm import tqdm

import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [ ]:
readfile_dir = "drive/My Drive/HAM10000/"
filename = "HAM10000_metadata.csv"

In [ ]:
df_metadata = pd.read_csv(readfile_dir+filename)

In [ ]:
list_image_id = list(df_metadata['image_id'].values)
print(len(list_image_id))

10015


In [ ]:
list_labels = list(df_metadata['dx'].values)
print(len(list_labels))

10015


In [ ]:
np_unique_classes = np.unique(np.array(list_labels))
print(np_unique_classes)

['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']


In [ ]:
list_label_int = []
for img_class in list_labels:
    if img_class == 'akiec': #np_unique_classes[0]:
        list_label_int.append(0)
    elif img_class == 'bcc':
        list_label_int.append(1)
    elif img_class == 'bkl':
        list_label_int.append(2)
    elif img_class == 'df':
        list_label_int.append(3)
    elif img_class == 'mel':
        list_label_int.append(4)
    elif img_class == 'nv':
        list_label_int.append(5)
    elif img_class == 'vasc':
        list_label_int.append(6)
print(len(list_label_int))

10015


In [ ]:
np_label_int = np.array(list_label_int)
np_label_int.shape

(10015,)

In [ ]:
df_imageset = pd.read_csv(readfile_dir+'HAM10000_images_all_in_one_Gray_maxPooling_3x3.csv', header=None)
df_imageset.shape

(10015, 30000)

In [ ]:
df_imageset

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,29960,29961,29962,29963,29964,29965,29966,29967,29968,29969,29970,29971,29972,29973,29974,29975,29976,29977,29978,29979,29980,29981,29982,29983,29984,29985,29986,29987,29988,29989,29990,29991,29992,29993,29994,29995,29996,29997,29998,29999
0,161,163,165,164,159,160,159,158,160,165,171,169,167,164,170,167,162,163,160,160,164,166,166,169,170,172,173,173,170,163,163,164,168,168,171,172,172,173,172,170,...,170,164,166,166,162,164,167,168,172,171,166,163,167,168,169,170,172,175,173,173,171,167,166,169,170,173,173,174,176,172,174,173,172,171,170,171,167,162,164,164
1,154,155,154,154,161,161,154,156,154,156,156,156,155,152,154,151,154,155,153,157,157,158,154,151,151,155,159,158,155,155,154,163,157,158,155,154,158,162,163,160,...,173,170,170,171,174,170,171,171,163,164,164,165,172,175,175,175,174,173,170,170,169,167,167,166,164,163,164,163,160,159,159,162,165,165,161,162,156,158,160,161
2,196,198,196,196,199,198,197,198,197,196,198,198,200,198,200,200,202,201,201,200,199,203,201,198,200,203,204,204,206,204,205,205,205,206,208,205,203,204,203,204,...,172,172,171,168,165,162,168,165,167,168,166,168,167,164,163,160,161,162,161,157,157,158,161,162,166,170,170,172,175,174,177,177,178,176,175,174,171,170,170,168
3,153,151,153,149,149,153,147,146,147,146,152,150,152,153,154,156,154,155,154,153,153,157,157,156,159,157,158,157,156,157,155,154,156,153,159,158,157,157,157,153,...,153,155,150,150,152,152,154,155,155,155,151,145,145,148,151,145,147,148,148,146,141,144,146,146,145,147,145,144,145,143,141,143,141,139,141,143,143,144,145,146
4,47,45,46,45,44,45,44,43,41,40,49,76,98,119,131,144,148,154,164,164,168,168,164,157,164,165,170,176,176,169,163,170,168,176,177,178,177,181,183,183,...,170,175,175,167,171,171,159,153,153,152,151,149,145,146,154,163,163,150,147,149,146,142,152,154,157,158,149,147,130,116,95,76,53,46,47,46,47,45,46,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10010,45,48,55,55,58,63,69,70,73,77,80,80,85,89,96,100,101,104,106,104,104,104,109,109,109,113,117,117,117,118,117,117,120,121,120,119,124,128,132,134,...,151,150,151,151,150,145,150,150,149,146,147,146,142,140,136,138,136,137,139,135,134,137,137,134,133,131,128,126,121,118,116,113,109,107,102,98,89,87,78,73
10011,204,204,203,206,207,212,212,213,213,214,216,216,217,218,216,215,216,220,223,222,223,222,221,224,223,222,222,224,223,222,224,223,225,224,225,227,227,226,225,226,...,231,230,228,230,228,226,226,228,231,229,224,223,221,221,220,220,220,218,217,218,219,217,219,219,218,216,214,216,218,219,217,216,218,217,218,218,218,214,214,214
10012,153,153,153,153,153,155,156,156,156,156,156,154,155,155,158,156,158,156,157,157,158,159,159,159,162,164,165,164,164,165,165,165,166,165,165,165,167,163,164,165,...,182,183,184,186,185,186,185,182,181,184,180,182,185,185,184,187,186,186,186,186,185,183,183,180,180,179,182,180,181,180,179,174,173,173,172,170,169,168,166,164
10013,61,66,71,72,76,76,77,79,79,77,82,86,88,90,93,95,97,100,101,99,99,101,98,99,101,105,108,108,108,109,108,109,109,108,110,110,110,111,113,115,...,110,112,111,114,112,114,112,112,112,111,111,110,109,105,104,101,100,98,99,98,98,99,99,96,95,94,94,92,91,90,89,87,87,85,83,81,78,74,73,71


In [ ]:
np_imageset = df_imageset.values
np_imageset.shape

(10015, 30000)

In [ ]:
list_imageset_reshaped = []
for i in range(np_imageset.shape[0]):
    reshape_data = np.reshape(np_imageset[i], (200,-1))
    list_imageset_reshaped.append(reshape_data)
print(len(list_imageset_reshaped))

10015


In [ ]:
np_imageset_reshaped = np.array(list_imageset_reshaped)
np_imageset_reshaped.shape

(10015, 200, 150)

In [ ]:
np_imageset_norm = np_imageset_reshaped/255

In [ ]:
split_index = int((len(np_imageset_norm)*10)/100)
print(split_index)

1001


In [ ]:
test_dataset = np_imageset_norm[0:split_index]
val_dataset = np_imageset_norm[len(np_imageset_norm)-split_index:len(np_imageset_norm)]
train_dataset = np_imageset_norm[split_index:len(np_imageset_norm)-split_index]

print('test_dataset.shape =',test_dataset.shape)
print('val_dataset.shape =',val_dataset.shape)
print('train_dataset.shape =',train_dataset.shape)

test_dataset.shape = (1001, 200, 150)
val_dataset.shape = (1001, 200, 150)
train_dataset.shape = (8013, 200, 150)


In [ ]:
test_dataset2 = np.expand_dims(test_dataset, axis = 3)
val_dataset2 = np.expand_dims(val_dataset, axis = 3)
train_dataset2 = np.expand_dims(train_dataset, axis = 3)

print('test_dataset2.shape =',test_dataset2.shape)
print('val_dataset2.shape =',val_dataset2.shape)
print('train_dataset2.shape =',train_dataset2.shape)

test_dataset2.shape = (1001, 200, 150, 1)
val_dataset2.shape = (1001, 200, 150, 1)
train_dataset2.shape = (8013, 200, 150, 1)


In [ ]:
test_labelset = np_label_int[0:split_index]
val_labelset = np_label_int[len(np_label_int)-split_index:len(np_label_int)]
train_labelset = np_label_int[split_index:len(np_label_int)-split_index]

print('test_labelset.shape =',test_labelset.shape)
print('val_labelset.shape =',val_labelset.shape)
print('train_labelset.shape =',train_labelset.shape)

test_labelset.shape = (1001,)
val_labelset.shape = (1001,)
train_labelset.shape = (8013,)


In [ ]:
test_labelset2 = np.expand_dims(test_labelset, axis = 1)
val_labelset2 = np.expand_dims(val_labelset, axis = 1)
train_labelset2 = np.expand_dims(train_labelset, axis = 1)

print('test_labelset2.shape =',test_labelset2.shape)
print('val_labelset2.shape =',val_labelset2.shape)
print('train_labelset2.shape =',train_labelset2.shape)

test_labelset2.shape = (1001, 1)
val_labelset2.shape = (1001, 1)
train_labelset2.shape = (8013, 1)


In [ ]:
num_classes = 7

In [ ]:
np_test_label_one_hot = tf.keras.utils.to_categorical(test_labelset2, num_classes)
np_val_label_one_hot = tf.keras.utils.to_categorical(val_labelset2, num_classes)
np_train_label_one_hot = tf.keras.utils.to_categorical(train_labelset2, num_classes)

print('np_test_label_one_hot.shape =',np_test_label_one_hot.shape)
print('np_val_label_one_hot.shape =',np_val_label_one_hot.shape)
print('np_train_label_one_hot.shape =',np_train_label_one_hot.shape)

np_test_label_one_hot.shape = (1001, 7)
np_val_label_one_hot.shape = (1001, 7)
np_train_label_one_hot.shape = (8013, 7)


In [ ]:
def res_block(X, filter, stage):
    
  # CONVOLUTIONAL BLOCK
  X_copy = X
  f1 , f2, f3 = filter

  # Main Path
  X = Conv2D(f1, (1,1), strides = (1,1), name ='res_'+str(stage)+'_conv_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = MaxPool2D((2,2))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_a')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_conv_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_b')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_c')(X)

  # Short path
  X_copy = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_conv_copy', kernel_initializer= glorot_uniform(seed = 0))(X_copy)
  X_copy = MaxPool2D((2,2))(X_copy)
  X_copy = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_conv_copy')(X_copy)

  # Add data from main and short paths
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

    
    
  # IDENTITY BLOCK 1
  X_copy = X
    
  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_1_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_a')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_1_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_b')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_1_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_1_c')(X)

  # Add both paths together (Note that we feed the original input as is hence the name "identity")
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

    
    
  # IDENTITY BLOCK 2
  X_copy = X

  # Main Path
  X = Conv2D(f1, (1,1),strides = (1,1), name ='res_'+str(stage)+'_identity_2_a', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_a')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f2, kernel_size = (3,3), strides =(1,1), padding = 'same', name ='res_'+str(stage)+'_identity_2_b', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_b')(X)
  X = Activation('relu')(X) 

  X = Conv2D(f3, kernel_size = (1,1), strides =(1,1),name ='res_'+str(stage)+'_identity_2_c', kernel_initializer= glorot_uniform(seed = 0))(X)
  X = BatchNormalization(axis =3, name = 'bn_'+str(stage)+'_identity_2_c')(X)

  # Add both paths together (Note that we feed the original input as is hence the name "identity")
  X = Add()([X,X_copy])
  X = Activation('relu')(X)

  return X

In [ ]:
train_dataset2.shape[1:]

(200, 150, 1)

In [ ]:
input_shape = train_dataset2.shape[1:] #(32,32,1) #np_train_dataset.shape #np.expand_dims(np_train_dataset, axis=1) #

# Input tensor shape
X_input = Input(input_shape)

# Zero-padding
X = ZeroPadding2D((1,1))(X_input)
#X = Input(input_shape)

# Stage #1
X = Conv2D(64, (5,5), strides= (1,1), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0))(X)
#X = Conv2D(64, np_train_dataset.shape[1:3], strides= (2,2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0), data_format='channels_first')(X)
#X = Conv2D(64, np_train_dataset.shape[1:3], strides= (2,2), name = 'conv1', kernel_initializer= glorot_uniform(seed = 0), data_format='channels_first')(X)
X = BatchNormalization(axis =3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
#print('ok1')
#X = MaxPooling2D((3,3), strides= (1,1))(X)
#print('ok2')
# Stage #2
#X = res_block(X, filter= [64,64,256], stage= 2)

# Stage #3
X = res_block(X, filter= [128,512,128], stage = 2)

#X = res_block(X, filter= [256,1024,256], stage = 3)

#X = res_block(X, filter= [512,1024,256], stage = 4)

#X = res_block(X, filter= [256,1024,256], stage = 5)

#X = res_block(X, filter= [64,64,256], stage= 6)

# Average Pooling
X = AveragePooling2D((2,2), name = 'Averagea_Pooling')(X)

# Final layer
X = Flatten()(X)
X = Dense(512, activation = 'relu')(X)
#X = Dropout(0.2)(X)
X = Dense(512, activation = 'relu')(X)
#X = Dropout(0.1)(X)
#X = Dense(1024, activation = 'relu')(X)
#X = Dense(512, activation = 'relu')(X)
X = Dense(num_classes, activation = 'softmax')(X)


model = Model( inputs= X_input, outputs = X)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 150, 1) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 202, 152, 1)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 198, 148, 64) 1664        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 198, 148, 64) 256         conv1[0][0]                      
______________________________________________________________________________________________

In [ ]:
#adam = tf.keras.optimizers.Adam(lr = 0.01) #, beta_1=0.9, beta_2=0.999, amsgrad=False
optimizer = tf.keras.optimizers.Adam(lr = 0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False) #, beta_1=0.9, beta_2=0.999, amsgrad=False
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

In [ ]:
# save the best model with least validation loss
checkpointer_name  = "weights_HAM10000_simple.hdf5"
checkpointer = ModelCheckpoint(filepath = readfile_dir+checkpointer_name, verbose = 1, save_best_only = True)

In [ ]:
start_time = time.time()
history = model.fit(train_dataset2, np_train_label_one_hot, 
                    shuffle=True, 
                    batch_size = 8, 
                    epochs= 15, 
                    #steps_per_epoch = 2,
                    #validation_split = 0.1, 
                    validation_data = (val_dataset2,np_val_label_one_hot),
                    callbacks=[checkpointer]
                    )
elapsed_time = time.time() - start_time
print("\nTime elapsed: ", elapsed_time)

Epoch 1/15
1002/1002 [==============================] - ETA: 0s - loss: 1.8586 - categorical_accuracy: 0.7039
Epoch 00001: val_loss improved from inf to 4.77790, saving model to drive/My Drive/HAM10000/weights_HAM10000_simple.hdf5
1002/1002 [==============================] - 448s 447ms/step - loss: 1.8586 - categorical_accuracy: 0.7039 - val_loss: 4.7779 - val_categorical_accuracy: 0.6693
Epoch 2/15
1002/1002 [==============================] - ETA: 0s - loss: 0.8738 - categorical_accuracy: 0.7518
Epoch 00002: val_loss did not improve from 4.77790
1002/1002 [==============================] - 439s 438ms/step - loss: 0.8738 - categorical_accuracy: 0.7518 - val_loss: 5.2602 - val_categorical_accuracy: 0.6693
Epoch 3/15
1002/1002 [==============================] - ETA: 0s - loss: 0.8409 - categorical_accuracy: 0.7528
Epoch 00003: val_loss did not improve from 4.77790
1002/1002 [==============================] - 439s 438ms/step - loss: 0.8409 - categorical_accuracy: 0.7528 - val_loss: 5.0069

KeyboardInterrupt: ignored

In [ ]:
result = model.evaluate(test_dataset2, np_test_label_one_hot)
print("Accuracy : {}, Loss: {}".format(result[1], result[0]))

32/32 [==============================] - 14s 424ms/step - loss: 13.3339 - categorical_accuracy: 9.9900e-04
Accuracy : 0.0009990009712055326, Loss: 13.333940505981445


In [ ]:
model_loaded = load_model(readfile_dir+checkpointer_name)

In [ ]:
result2 = model_loaded.evaluate(test_dataset2, np_test_label_one_hot)
print("Accuracy : {}, Loss: {}".format(result2[1], result2[0]))

32/32 [==============================] - 13s 403ms/step - loss: 5.5483 - categorical_accuracy: 9.9900e-04
Accuracy : 0.0009990009712055326, Loss: 5.548250675201416


In [ ]:
#Confution Matrix and Classification Report
Y_pred = model_loaded.predict_generator(test_dataset2, len(test_dataset2))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_labelset2, y_pred))

Instructions for updating:
Please use Model.predict, which supports generators.
Confusion Matrix
[[   0 1000]
 [   0    1]]


In [ ]:
test_labelset2

array([[2],
       [2],
       [2],
       ...,
       [2],
       [2],
       [2]])

In [ ]:
# Precision [TP/TP+FP] = The ratio of correctly predicted positive observations to the total predicted positive observations.
# Recall (Sensitivity) [TP/TP+FN] = The ratio of correctly predicted positive observations to the all observations in actual class - 'yes'.
# F1 score [F1 Score = 2*(Recall * Precision) / (Recall + Precision)] = The weighted average of Precision and Recall.
# Support = The number of samples of the true response that lie in that class.
print('Classification Report:')
#target_names = ['Mono', 'Di'] # not ['Di', 'Mono']
print(classification_report(test_labelset2, y_pred)) #, target_names=target_names))

Classification Report:
              precision    recall  f1-score   support

           2       0.00      0.00      0.00      1000
           5       0.00      1.00      0.00         1

    accuracy                           0.00      1001
   macro avg       0.00      0.50      0.00      1001
weighted avg       0.00      0.00      0.00      1001



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print('Classification Report')
target_names = ['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']
print(classification_report(test_labelset2, y_pred, target_names=target_names))

In [ ]:
# Getting the model history keys 
history.history.keys()

In [ ]:
# plot the training artifacts

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'upper right')
plt.show()

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy','val_accuracy'], loc = 'upper right')
plt.show()